In [107]:
import pandas as pd
import math
import pickle

In [108]:
df = pd.read_csv( "/kaggle/input/annots/VIA_annotations.csv", header =1)

In [109]:
df.head()

,# CSV_HEADER = metadata_id,file_list,temporal_segment_start,temporal_segment_end,metadata
0,1_c3ruk3yo,"[""free.MP4""]",1.008,1.707,"{""subtitle"":""""}"
1,1_JshBo2aX,"[""free.MP4""]",1.707,2.660,"{""subtitle"":""""}"
2,1_tTqXX2us,"[""free.MP4""]",2.660,3.800,"{""subtitle"":""""}"
3,2_uXIq8HXz,"[""free (2).MP4""]",0.514,1.082,"{""subtitle"":""""}"
4,2_eXqfdxsH,"[""free (2).MP4""]",1.082,2.113,"{""subtitle"":""""}"


In [110]:
def trunc(stri):
    strn = stri[2:-2]
    return(strn)
    
trunc("[\"free.MP4\"]")

'free.MP4'

In [111]:
modified_column = df['file_list'].apply(trunc)
df["file_list"] = modified_column

In [112]:
df.head()

,# CSV_HEADER = metadata_id,file_list,temporal_segment_start,temporal_segment_end,metadata
0,1_c3ruk3yo,free.MP4,1.008,1.707,"{""subtitle"":""""}"
1,1_JshBo2aX,free.MP4,1.707,2.660,"{""subtitle"":""""}"
2,1_tTqXX2us,free.MP4,2.660,3.800,"{""subtitle"":""""}"
3,2_uXIq8HXz,free (2).MP4,0.514,1.082,"{""subtitle"":""""}"
4,2_eXqfdxsH,free (2).MP4,1.082,2.113,"{""subtitle"":""""}"


In [113]:
annotation=df.iloc[1:,1:-1]
annotation = annotation.reset_index()
annotation = annotation.iloc[0:, 1:]

annotation.head()

,file_list,temporal_segment_start,temporal_segment_end
0,free.MP4,1.707,2.660
1,free.MP4,2.660,3.800
2,free (2).MP4,0.514,1.082
3,free (2).MP4,1.082,2.113
4,free (2).MP4,2.113,2.847


In [114]:
labelDict = {}
for index, row in annotation.iterrows():
    if row['file_list'] not in labelDict:
        labelDict[row['file_list']] = []
        
    label = 0
    start,stop=float(row['temporal_segment_start']), float(row['temporal_segment_end'])
    labelDict[row['file_list']].append({"label":label, "start":start, "stop":stop})

In [115]:
for key in labelDict.keys():
    labelDict[key] = sorted(labelDict[key], key=lambda i: i['start']) 
    for i in range(len(labelDict[key])-1):
        label = 1
        a,b = labelDict[key][i]['stop'], labelDict[key][i+1]['start']
        if (a<b):
            start, stop = a, b
        else:
            start, stop = b, a
            labelDict[key][i] = {"label":0, 'start':labelDict[key][i]['start'], 'stop':b}
            labelDict[key][i+1] = {"label":0, 'start':a, 'stop':labelDict[key][i+1]['stop']}

        labelDict[key].append({"label":label, "start":start, "stop":stop})
    labelDict[key] = sorted(labelDict[key], key=lambda i: i['start']) 

In [116]:
for key in labelDict.keys():
    for i in range(len(labelDict[key])):
        labelDict[key][i] = {"label":labelDict[key][i]['label'],'start':labelDict[key][i]['start']*25, 'stop':labelDict[key][i]['stop']*25}

In [117]:
frameDict = {}
for key in labelDict.keys():
    frameList = {}
    firstFrame = int(labelDict[key][0]["start"])
    lastFrame = int(labelDict[key][-1]["stop"])
    for annotDict in labelDict[key]:
        start = int(annotDict["start"])
        stop = int(annotDict["stop"])
        start = math.floor(start)
        stop = math.floor(stop)

        if(start == firstFrame):
            start+=1
        if(stop == lastFrame):
            stop+=1

        for i in range(start, stop):
            frameList[i] = annotDict["label"]
    frameDict[key] = frameList

    

In [118]:
for key in frameDict.keys():
    for num in frameDict[key].keys():
        if (num != 1) and (num != len(frameDict[key])):
            if (frameDict[key][num] == 1) and (frameDict[key][num-1] == 0) and (frameDict[key][num+1] == 0):
                frameDict[key][num-1] = 1
                frameDict[key][num+1] = 1

In [119]:
len(frameDict)

176

In [120]:
for key in frameDict.keys():
    List = [num for num in frameDict[key].keys()]
    minFrame = min(List)
    frameDict[key][minFrame-1] = 1

In [49]:
with open("/kaggle/working/frameDict.pkl", "wb") as f:
    pickle.dump(frameDict,f)
    

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/frameDict.pkl'